Imports

In [1]:
import os
import cv2
import random
from tqdm import tqdm
import glob
from pathlib import Path
import shutil
from os import listdir
from os.path import join, isfile
from deepface import DeepFace
import numpy as np
from PIL import Image 

Influencer names

In [3]:
influencers = ['Airrack', 'BellaPoarch', 'Larry', 'LexiHensler', 'LoganPaul', 'MarkRober', 'MrBeast', 'PiersonWodzynski', 'Preston', 'Quackity', 'RosannaPansino', 'RyanTrahan', 'SoffieDossi', 'ZachKing', 'Zhc' ]

Renaming folders

In [ ]:
def rename(folder):

    for count, filename in enumerate(os.listdir(folder)):
        dst = f"Image{str(count)}.jpg"
        src =f"{folder}/{filename}"  
        dst =f"{folder}/{dst}"
        
        os.rename(src, dst)
        

List all subfolders in YouTubeFaces/frame_images_DB

In [ ]:
subfolders = [ f.path for f in os.scandir("/YouTubeFaces/frame_images_DB/") if f.is_dir() ]

all_names = []
for name in subfolders:
    name = name.split("/")
    name = name[-1]
    all_names.append(name)

List all subfolders in YouTubeFaces/frame_images_DB with number of sub directories

In [5]:
dcount = []

for i in range(len(subfolders)):
    num_of_sub_dirs = 0
    current_folder = subfolders[i]
    path = Path(current_folder)
    for f in path.glob('*'): 
        if f.is_dir():
            num_of_sub_dirs += 1
        
    dcount.append(str(path) + ' ' + str(num_of_sub_dirs))


In [49]:
allpaths = []
with open('allpaths.txt', 'r') as f:
    for line in f:
        line = line.strip()
        allpaths.append(line)

all_names_2ormore = []
for name in allpaths:
    name = name.split('/')
    name = name[-1]
    all_names_2ormore.append(name)
print(all_names_2ormore[:5])
print(len(all_names_2ormore))



['Aaron_Eckhart', 'Aaron_Sorkin', 'Aaron_Tippin', 'Abba_Eban', 'Abdel_Aziz_Al-Hakim']
1004


All persons that appear in 1 video

In [ ]:
all_names #1595 names
all_names_2ormore #1004 names

all_names_1_dir = []
path = '/YouTubeFaces/frame_images_DB/'
for name in all_names:
    if name not in all_names_2ormore:
        name = path + name
        all_names_1_dir.append(name)

print(all_names_1_dir[:5])
print(len(all_names_1_dir))

all_names_1_dirdst = []
pathdst = 'Youtube/1video/'

for name in all_names_1_dir:
    name = name.split('/')
    name = name[-1]
    name = pathdst + name
    all_names_1_dirdst.append(name)

print(all_names_1_dirdst[:5])

Create directories

In [58]:
# Create directory

def createDirectory(dirnamesList):
    for i in range(len(dirnamesList)):
        dirName = dirnamesList[i]
        try:
        # Create target Directory
            os.mkdir(dirName)
            print("Directory " , dirName ,  " Created ") 
        except FileExistsError:
            print("Directory " , dirName ,  " already exists")

Copy from source to destination

In [ ]:
# src all_names_1_dir, dst all_names_1_dirdst
for i, j in zip(range(len(all_names_1_dir)), range(len(all_names_1_dirdst))):
    current_src = allpaths[i] 
    print('Current src: ', current_src)
    current_dst = all_names_1_dirdst[j]
    print('Current dst: ', current_dst)
    for root, dirs, files in os.walk(current_src):  #starting directory
        for file in files:
            path_file = os.path.join(root, file)
            shutil.copy2(path_file, current_dst) #destination dir 


Select random images

In [1]:
def randomImagePicker(sourceList, destinationList, numberOfImages):

    for src, dst in tqdm(zip(range(len(sourceList)), range(len(destinationList)))):
        src_dir = sourceList[src]
        dst_dir = destinationList[dst]
        #print('src: ', src_dir)
        #print('dst: ', dst_dir)
        file_list = os.listdir(src_dir)
        #print('file list: ', file_list)
        #print(len(file_list))
        random_list = random.sample(file_list, numberOfImages)
        #print('random list: ', random_list)

        for i in range(len(random_list)):
            current_file = random_list[i]
            #print('src: ',src_dir + "/" + current_file)
            #print('dst ',dst_dir  + "/" + current_file)
            shutil.copy(src_dir + "/" + current_file, dst_dir + "/" + current_file)


Resize images

In [ ]:
def resize(dirlist, width, height):
    for i in range(len(dirlist)):
        current_dir = dirlist[i]
        #print('current_dir', current_dir)

        images = Path(current_dir).glob("*.jpg")
        image_strings = [str(p) for p in images]

        for image in image_strings:
            image = image.replace('\\','/')
            #print('current img before reading', image)
            img = cv2.imread(image, cv2.IMREAD_UNCHANGED) # read image
            #print('image in imread function', img)
            resize_width = width
            resize_height = height
            dim = (resize_height, resize_width)

            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

            cv2.imwrite(image, resized)

Convert to grayscale

In [ ]:
def to_gray(pathlist):
    for path in pathlist:
        files = [f for f in listdir(path) if isfile(join(path,f))] 
        #print(path, files)

        for image in files:
            img = os.path.join(path, image)
            img = img.replace('\\','/')
            #print(img)
            try:
                current_img_gray = cv2.imread(img)
                gray = cv2.cvtColor(current_img_gray, cv2.COLOR_BGR2GRAY)
                cv2.imwrite(img, gray)
            except:
                print ("{} is not converted".format(img))


Haar facedetection


In [42]:
#take from dirnames30, dirnames50 and dirnames70
#save in cropped30, cropped50, cropped70

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def haar_facedetection(srcpath, dstpath):
    #take paths from srcpath, detect face and save face to dstpath
    count = 0
    for src, dst in zip(srcpath, dstpath):
        currentsrc = src
        currentdst = dst
        for filepath in glob.iglob(currentsrc + '/*.jpg'):
            filepath = filepath.replace('\\','/')
            filename = filepath.split('/')[-1]

            img = cv2.imread(filepath)
            faces=face_cascade.detectMultiScale(img, 1.1, 4)

            savepath = currentdst + '/' + filename
            if faces == ():
                no_face = Image.open(filepath)
                no_face.save(savepath)
                count += 1
                
            for x, y, w, h in faces:
                face = img[y:y+h, x:x+w]
                cv2.imwrite(savepath, face)
                x=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
    return 'Could not detect: ' + str(count) + ' faces'

Dlib facedetection


In [20]:
def dlib_facedetection(srcpath, dstpath):
    for src, dst in zip(srcpath, dstpath):
            currentsrc = src
            currentdst = dst
            for filepath in glob.iglob(currentsrc + '/*.jpg'):
                filepath = filepath.replace('\\','/')
                filename = filepath.split('/')[-1]
                img = DeepFace.detectFace(filepath, detector_backend= 'dlib', enforce_detection=False)
                detected_face = img * 255
                savepath = currentdst + '/' + filename
                cv2.imwrite(savepath, detected_face[:, :, ::-1])